
# Input data

## Weather forcing

The first example uses data from ECMWF ERA5 obtained from [CDS](https://cds.climate.copernicus.eu/#!/home)

A common format for multidimensional data is [NetCDF](https://www.unidata.ucar.edu/software/netcdf/).

An excellent python package for working with NetCDF in Python is [Xarray](http://xarray.pydata.org/en/stable/).

In [ ]:
! pip install netcdf4 xarray

In [ ]:
import xarray as xr

ds = xr.open_dataset("data/era5_small.nc")
ds

In [ ]:
ds.t2m

To select the first timestep, we can use the `xarray.Dataset.isel` method.

In [ ]:
ds0 = ds.isel(time=0)
ds0

In [ ]:
ds0.t2m.plot()

## Pressure

Atmospheric pressure comes in different flavors.

`Surface pressure` is the pressure at ground level, i.e. following the terrain.

In the following plot you can clearly see the decrease in surface pressure with altitude in areas with mountains.

This is *NOT* the variable used by MIKE 21 FM HD.

In [ ]:
ds0.sp.plot()

MIKE 21 FM HD expects the atmospheric pressure reduced to Mean Sea Level.

In the following plot the pressure field has a much smaller range, since the variation with altitude have been removed.

In [ ]:
ds0.msl.plot()

## Wind

In [ ]:
ds0.u10

## Convert to dfs2

### Sub-region

In [ ]:
ds_aoi = ds.sel(longitude=slice(10,30), 
                latitude=slice(60,45)) # N -> S
ds_aoi

In [ ]:
ds_aoi.isel(time=0).u10.plot(cmap='jet')

In [ ]:
import pandas as pd

time = pd.DatetimeIndex(ds.time)
time

In [ ]:
data = [ds_aoi.msl.values / 100.0, # Pa -> hPa
        ds_aoi.u10.values,
        ds_aoi.v10.values,
        ds_aoi.t2m.values - 273.15] # K -> °C

In [ ]:
from mikeio.eum import EUMType, EUMUnit, ItemInfo

items = [
        ItemInfo("MSLP", EUMType.Pressure, EUMUnit.hectopascal),
        ItemInfo("U 10m", EUMType.Wind_Velocity, EUMUnit.meter_per_sec),
        ItemInfo("V 10m", EUMType.Wind_Velocity, EUMUnit.meter_per_sec),
        ItemInfo("Temperature", EUMType.Temperature, EUMUnit.degree_Celsius)
]

In [ ]:
from mikeio import Dataset

my_ds = Dataset(data=data, time=time, items=items)
my_ds

In [ ]:
ds_aoi.longitude.values[0], ds.longitude.values[-1]

In [ ]:
dx = ds_aoi.longitude.values[1] - ds_aoi.longitude.values[0]
dx

In [ ]:
ds_aoi.latitude.values[0], ds_aoi.latitude.values[-1]

In [ ]:
dy = ds_aoi.latitude.values[0] - ds_aoi.latitude.values[1] # dx, dy should be positive!
dy

In [ ]:
type(data[0])

The dimensions for at  DFS2 are expected to be (t, y, x), which matches the ones used by this dataset.

This convention is recommended by the [CF convention](http://cfconventions.org/cf-conventions/cf-conventions.html#dimensions)

In [ ]:
ds.Conventions

In [ ]:
ds.u10.dims

In [ ]:
data[0].shape

In [ ]:
lon_origin = ds_aoi.longitude.values[0]
lon_origin

In [ ]:
lat_origin = ds_aoi.latitude.values[-1] # Southern-most point
lat_origin

In [ ]:
# This dataset North-South orientation matches the expected, but for other dataset you might need to flip the data upside-down
# It can be done like this
# my_ds = my_ds.flipud()

In [ ]:
from mikeio import Dfs2

orientation = 0.0

dfs = Dfs2()
dfs.write("era5_aoi.dfs2", data=my_ds, coordinate=["LONG/LAT", lat_origin, lon_origin, orientation], dx=dx, dy=dy)


Screenshot of U 10m from MIKE Zero

![](images/era5_u10_dfs2.png)

## Unstructured grid forcing

E.g. roughness map

In [ ]:
from mikeio import Mesh, Dfsu

msh = Mesh("data/FakeLake.dfsu")
msh.plot()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

z = np.linspace(0,-30)
manning = 32 + (40 - 32)*(1 - np.exp(z/10))

plt.plot(z, manning)
plt.xlim(0,-30)
plt.ylabel("Manning")
plt.xlabel("Depth (m)")
plt.title("This is only an example...")

In [ ]:
ze = msh.element_coordinates[:,2]

In [ ]:
me = 32 + (40 - 32)*(1 - np.exp(ze/10))

In [ ]:
me2d = np.atleast_2d(me) # Introduce time axis
me2d.shape

In [ ]:
from mikeio.eum import ItemInfo
from mikeio import Dataset

my_ds = Dataset(data=[me2d],
                time="2000-1-1", # For static data, this is not used by MIKE 21
                items=[ItemInfo("Roughness", EUMType.Mannings_M)])
my_ds

In [ ]:
dfs = Dfsu("data/FakeLake.dfsu") # use this file as template
dfs.write("manning.dfsu", data=my_ds)

In [ ]:
dfs = Dfsu("manning.dfsu")
dfs

In [ ]:
ds = dfs.read()
dfs.plot(ds["Roughness"][0], title="Manning")